# MPI

MPI (Message Passing Interface) is a standard for writing parallel programs that can run on a distributed memory system. It is widely used in the HPC (High Performance Computing) community. There are implementations of MPI for many programming languages, including C, C++, Fortran, and Python. Much of the functionality and many of the commands we will see here will be similar in other languages. 

There are multiple implementations of MPI, including [OpenMPI](https://www.open-mpi.org/), [MPICH](https://www.mpich.org/), and [Intel MPI](https://www.intel.com/content/www/us/en/developer/tools/oneapi/mpi-library.html). If you are running this notebook in a Github Codespace, MPICH will already be installed. If you want to install MPICH on your local system you can follow the instructions [here](https://www.mpich.org/downloads/).

We will also be using the `mpi4py` package which is a Python wrapper for MPI. This is already installed if you'r running this notebook in a GitHUb Codespace, or you can install it locally using pip:

```bash
pip install mpi4py
```

Unlike the other methods we have seen so far, we run MPI from the terminal using the `mpiexec` command. We can run a Python script named `python_script.py` using MPI like this:

```bash
mpiexec -n 4 python python_script.py
```
In this command, the `-n 4` flag tells MPI to run the script using 4 processes. The text `python python_script.py` tells which command MPI should be running on each process. This will run the script 4 times in parallel.

## Ranks

Each copy of the code will be executed on its own process and will be identified by its "rank". The rank is a unique integer identifier for each process which can be accessed using the `mpi4py.MPI.COMM_WORLD.Get_rank()` function. We can also get the number of ranks using the method `Get_rank()`. The code below shows how to access the rank:

```python
# Run this script with the terminal command `mpiexec -n 4 python get_rank.py`

import mpi4py.MPI as MPI

# Get a reference to the current MPI.COMM_WORLD communicator
comm = MPI.COMM_WORLD

# Get the total number of ranks in the communicator
n_rank = comm.Get_size()

# Get the rank of the current process
rank = comm.Get_rank()

# Print the rank of the current process
print(f'This script is being run by Rank {rank} out of {n_rank} total ranks')
```

This code can be found in the file `04_mpi_scripts/get_rank.py`. To run this command we will need to change directory in the terminal using the command:

```bash
cd 04_mpi_scripts
```

and run the script using MPI using the command:

```bash
mpiexec -n 4 python 04_mpi_scripts/get_rank.py
```

## Communicating Between Ranks

In the above example, we access the variable `MPI.COMM_WORLD` which references a communicator. We can use a communicator to send messages between the processes it contains. The communicator `MPI.COMM_WORLD` contains all the processes that are running the script. It is possible to create other communicators which contain only a subset of the processes, which can be useful for more complex parallel programs, but we won't be covering that here.

We can send messages between ranks using the `send` and `recv` methods of the communicator. The code below shows how to send a message from rank 0 to rank 1:

```python
from mpi4py import MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # If we're in rank 0, send a dictionary to rank 1
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1)
elif rank == 1:
    # If we're in rank 1, receive the dictionary from rank 0
    data = comm.recv(source=0)
    print(rank, data)
```

In the example above the `send` method is used to send a dictionary from rank 0 to rank 1. By including it in the if-block, we make sure it is only called by the rank 0. The first argument to `send` is the data to be sent. We also specify the destination rank so the message can be sent to rank 1. The `recv` method is called from rank 1 to receive the message from rank 0. The `source` argument specifies the rank of the process that sent the message. The data which is received is saved into the variable `data` and then printed.

Both `send` and `recv` block, meaning that the program will wait at the `send` line until the message has been received by the destination rank, and will wait at the `recv` line until the message has been sent by the source rank. This means we need to plan carefully to make sure that the program doesn't get deadlocked. For example, the code below will deadlock as both ranks are waiting for the other to receive a message:

```python
from mpi4py import MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # Send a message to rank 1
    comm.send("Hello from rank 0", dest=1)
    # Receive the message from rank 1
    data = comm.recv(source=1)
elif rank == 1:
    # Send a message to rank 0
    comm.send("Hello from rank 1", dest=0)
    # Receive the message from rank 0
    data = comm.recv(source=0)
```

This code can be found in the file [`04_mpi_scripts/deadlock.py`](04_mpi_scripts/deadlock.py) and should be run with two processes.
